# TAL - Classification de dépêches d’agence avec NLTK

In [1]:
# importing modules
import nltk
import string
import collections

from nltk.corpus import reuters
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from nltk.metrics.scores import (precision, recall, f_measure)

from random import shuffle

In [2]:
# Extract fileids from the reuters corpus
fileids = reuters.fileids()
documents = []
# Loop through each file id and collect each files categories and tokenized words
for file in fileids:
    words = reuters.words(file)
    documents.append((words, reuters.categories(file)))

shuffle(documents)
documents[0]

(['NORTH', 'EAST', '&', 'lt', ';', 'NEIC', '>', 'MAY', ...], ['earn'])

Nous avons remarqué que dans la tokenisation des mots du corpus, le mots `U.S` est séparé en trois tokens distinct, `U`, `.` et `S`. Nous avon estimé que dans le cadre de ce labo, cela ne devrait pas causer trop de problèmes et nous avons donc laissé cette séparation.

## Classifieur binaire

Pour la classification des documents, nous avons décidé d'utiliser la fréquence des mots. Nous avons donc commencé par déterminer la fréquence de **TOUT** les mots du dataset (i.e. tout les documents), puis les `2000` mots les plus fréquents sont retourné.

> Note: La limite de la fréquence des mots que la fonction retourne est paramètrable.

In [3]:
def document_features(document, word_frequence):
    document_words = set(document)
    features = {}
    for word in word_frequence:
        features['contains({})'.format(word)] = (word in document_words)
    return features

def most_freq_words(documents, limit=2000):
    all_words = nltk.FreqDist(w
        for document in documents
        for w in document[0]
    )
    return list(all_words)[:limit]

In [4]:
def create_dataset(documents, tag, feature_extractor, **kwargs):
    if 'to_lower' in kwargs and kwargs['to_lower']:
        documents = list(map(lambda d: (list(map(str.lower, d[0])), d[1]), documents))

    if 'lemmatizer' in kwargs:
        lemmatizer = kwargs['lemmatizer']
        documents = list(map(lambda d: (list(map(lemmatizer.lemmatize, d[0])), d[1]), documents))
    
    if 'stopwords' in kwargs:
        stopwords = set(kwargs['stopwords'])
        documents = list(map(
            lambda d: (
                list(filter(lambda w: not w.lower() in stopwords and w[0].isalnum(), d[0])), 
                d[1]
            ), documents))
        
    analyzer_res = []
    if 'analyzer' in kwargs:
        analyzer_res = kwargs['analyzer'](documents)

    dataset = []
    for document in documents:
        dataset.append((feature_extractor(document[0], analyzer_res), tag in document[1]))
    
    shuffle(dataset)
    return dataset

def split_dataset(dataset):
    split_ratio = 0.6
    split_ratio2 = 0.8
    
    split = int(len(dataset) * split_ratio)
    split2 = int(len(dataset) * split_ratio2)

    return (dataset[:split], dataset[split:split2], dataset[split2:])

In [5]:
def best_classifier(documents, tag, dataset_creator, hyperparams):
    print('Finding best classifier for {}'.format(tag))
    print('----------')

    best = (None, 0.0)
    for hyperparam in hyperparams:
        dataset = dataset_creator(documents, tag, **hyperparam)
        train_set, test_set, dev_set = split_dataset(dataset)
        classifier = nltk.NaiveBayesClassifier.train(train_set)
        acc = nltk.classify.accuracy(classifier, dev_set)
        
        if acc > best[1]:
            best = (classifier, acc)
        
        print('Accuracy using "{}": {:.2f}%'.format(hyperparam['title'], acc*100))
    return (best[0], test_set)

### Combinaison des différents hyperparamètres

Pour les hyperparamètres, nous avons choisi d'utiliser de supprimer les stopwords, la lemmatisation et de tout mettre en minuscules. En plus de tester chaque hyperparamètre indépendamment, nous avons aussi testé d'appliquer plusieurs hyperparamètre en même temps (e.g. lemmatiser et supprimer les stopwords).

In [17]:
hyperparams = [
    {
        'title': 'To lower: no, Lemmatize: no, No stopwords: no',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
    },
    {
        'title': 'To lower: yes, Lemmatize: no, No stopwords: no',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'to_lower': True,
    },
    {
        'title': 'To lower: no, Lemmatize: yes, No stopwords: no',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'lemmatizer': WordNetLemmatizer(),
    },
    {
        'title': 'To lower: no, Lemmatize: no, No stopwords: yes',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'stopwords': stopwords.words('english'),
    },
    {
        'title': 'To lower: yes, Lemmatize: no, No stopwords: yes',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'to_lower': True,
        'stopwords': stopwords.words('english'),
    },
    {
        'title': 'To lower: no, Lemmatize: yes, No stopwords: yes',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'lemmatizer': WordNetLemmatizer(),
        'stopwords': stopwords.words('english'),
    },
    {
        'title': 'To lower: yes, Lemmatize: yes, No stopwords: no',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'to_lower': True,
        'lemmatizer': WordNetLemmatizer(),
    },
    {
        'title': 'To lower: yes, Lemmatize: yes, No stopwords: yes',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'to_lower': True,
        'stopwords': stopwords.words('english'),
        'lemmatizer': WordNetLemmatizer(),
    },
]

### Classification des documents `money-fx`

Si l'on compare chaque hyperparamètres seul, on peut voir que la mise en minuscule est légèrement meilleur que les autres. Le fait de combiner la mise en minuscule avec les autres hyperparamètres améliore les scores. Et finalement, on remarque que combiner les trois hyperparamètres améliore le score (accuracy) du classifieur d'environ 3%.

>Résultats lors de l'écriture des commentaires:
>
>```
>Finding best classifier for money-fx
>----------
>Accuracy using "To lower: no, Lemmatize: no, No stopwords: no": 87.72%
>Accuracy using "To lower: yes, Lemmatize: no, No stopwords: no": 88.74%
>Accuracy using "To lower: no, Lemmatize: yes, No stopwords: no": 87.86%
>Accuracy using "To lower: no, Lemmatize: no, No stopwords: yes": 90.73%
>Accuracy using "To lower: yes, Lemmatize: no, No stopwords: yes": 90.55%
>Accuracy using "To lower: no, Lemmatize: yes, No stopwords: yes": 90.64%
>Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: no": 87.53%
>Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: yes": 91.71%
>```

In [8]:
classifier_moneyfx, moneyfx_testset = best_classifier(documents, 'money-fx', create_dataset, hyperparams)

Finding best classifier for money-fx
----------
Accuracy using "To lower: no, Lemmatize: no, No stopwords: no": 87.72%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: no": 88.74%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: no": 87.86%
Accuracy using "To lower: no, Lemmatize: no, No stopwords: yes": 90.73%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: yes": 90.55%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: yes": 90.64%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: no": 87.53%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: yes": 91.71%


### Classification des documents `wheat`

Si l'on compare chaque hyperparamètres seul, on peut voir que cette fois, c'est la suppression des stopwords est meilleur que les autres. Le fait de combiner la suppression des stopwords avec les autres hyperparamètres améliore les scores. Et finalement, on remarque que combiner les trois hyperparamètres améliore le score (accuracy) du classifieur, mais qu'il n'est pas le meilleur (comme pour le classifieur `money-fx`). Ici le meilleur classifieur est celui qui combine la supression des stopwords et la mise en minuscule.

>Résultats lors de l'écriture des commentaires:
>
>```
>Finding best classifier for money-fx
>----------
>Accuracy using "To lower: no, Lemmatize: no, No stopwords: no": 89.99%
>Accuracy using "To lower: yes, Lemmatize: no, No stopwords: no": 89.76%
>Accuracy using "To lower: no, Lemmatize: yes, No stopwords: no": 89.99%
>Accuracy using "To lower: no, Lemmatize: no, No stopwords: yes": 93.51%
>Accuracy using "To lower: yes, Lemmatize: no, No stopwords: yes": 94.86%
>Accuracy using "To lower: no, Lemmatize: yes, No stopwords: yes": 93.23%
>Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: no": 90.59%
>Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: yes": 94.35%
>```

In [9]:
classifier_wheat, wheat_testset = best_classifier(documents, 'wheat', create_dataset, hyperparams)

Finding best classifier for wheat
----------
Accuracy using "To lower: no, Lemmatize: no, No stopwords: no": 89.99%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: no": 89.76%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: no": 89.99%
Accuracy using "To lower: no, Lemmatize: no, No stopwords: yes": 93.51%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: yes": 94.86%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: yes": 93.23%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: no": 90.59%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: yes": 94.35%


### Classification des documents `gold`

Pour ce classifieur nous nous retrouvons plus ou moins dans la même situation que le classifieur `wheat`. C'est-à-dire que le meilleur classifieur avec un seul hyperparamètre et aussi celui qui supprime les stopwords. Sauf que quand nous combinons les hyperparamètre, il y a certes une amélioration des classifieurs qui utilise les deux autres hyperparamètres **mais**, aucune combinaison n'est meilleur que le classifieur avec seulement la suppression des stopwords.

>Résultats lors de l'écriture des commentaires:
>
>```
>Finding best classifier for money-fx
>----------
>Accuracy using "To lower: no, Lemmatize: no, No stopwords: no": 93.79%
>Accuracy using "To lower: yes, Lemmatize: no, No stopwords: no": 95.09%
>Accuracy using "To lower: no, Lemmatize: yes, No stopwords: no": 96.71%
>Accuracy using "To lower: no, Lemmatize: no, No stopwords: yes": 98.75%
>Accuracy using "To lower: yes, Lemmatize: no, No stopwords: yes": 98.42%
>Accuracy using "To lower: no, Lemmatize: yes, No stopwords: yes": 98.05%
>Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: no": 98.19%
>Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: yes": 98.56%
>```

In [10]:
classifier_gold, gold_testset = best_classifier(documents, 'gold', create_dataset, hyperparams)

Finding best classifier for gold
----------
Accuracy using "To lower: no, Lemmatize: no, No stopwords: no": 93.79%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: no": 95.09%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: no": 96.71%
Accuracy using "To lower: no, Lemmatize: no, No stopwords: yes": 98.75%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: yes": 98.42%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: yes": 98.05%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: no": 98.19%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: yes": 98.56%


Si l'on compare les trois classifieurs n'utilise pas les même hyperparamètres, mais l'on remarque que le classifieur `money-fx` est moins bon que les deux autres. Nous avons comparé la taille des datasets et nous avons remarqué que celui pour `money-fx` est beaucoup plus grands que les deux autres (717 `money-fx` 283 `wheat` et 124 `gold`). Cette différence en taille explique la différence de qualité dans les classifieurs, étant donnée que les classifeurs `wheat` et `gold` on des dataset plus petit il y a une plus grande chance que les documents de leur dataset contienne les mots les plus fréquents.

In [14]:
def ref_test_sets(testset, classifier):
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)

    for i, (feats, label) in enumerate(testset):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)

    return refsets, testsets

In [31]:
moneyfx_refsets, moneyfx_testsets = ref_test_sets(moneyfx_testset, classifier_moneyfx)
wheat_refsets, wheat_testsets = ref_test_sets(wheat_testset, classifier_wheat)
gold_refsets, gold_testsets = ref_test_sets(gold_testset, classifier_gold)

En regardant la précision, le rappel et la F-Mesure de tout nos classifieurs, on remarque qu'ils ne sont pas précis du tout. C'est-à-dire qu'il n'arrive pas à correctement classifier les documents du type que l'on souhaite classifier. Par contre, si l'on regarde le rappel, on voit qu'ils ont tous de très bon score (> 80%), ce qui veut dire qu'ils sont capables de bien classifier les documents qui ne sont pas ceux que l'on souhaite classifier. Cette différence drastique est normale, car nous avons une distribution qui n'est pas bien répartie (i.e. il y a plus de documents qui ne font pas parti de la classe). Et donc pour vraiment évaluer la qualité de notre classifieur, il faut regarder le F-Mesure. Et là, on voit que nos classifieurs ont un score d'environ 50%, ce qui veut dire qu'ils sont bof.

>Résultats lors de l'écriture des commentaires:
>
>```
>Money-fx:
>---------
>Precision: 0.3639344262295082
>Recall: 0.8283582089552238
>F-mesure: 0.5056947608200456
>
>Wheat:
>---------
>Precision: 0.32098765432098764
>Recall: 0.9285714285714286
>F-mesure: 0.4770642201834862
>
>Gold:
>---------
>Precision: 0.5306122448979592
>Recall: 0.896551724137931
>F-mesure: 0.6666666666666666
>```

In [32]:
print('Money-fx:')
print('---------')
print('Precision:', precision(moneyfx_refsets[True], moneyfx_testsets[True]))
print('Recall:'   , recall(moneyfx_refsets[True], moneyfx_testsets[True]))
print('F-mesure:' , f_measure(moneyfx_refsets[True], moneyfx_testsets[True]))

print()

print('Wheat:')
print('---------')
print('Precision:', precision(wheat_refsets[True], wheat_testsets[True]))
print('Recall:'   , recall(wheat_refsets[True], wheat_testsets[True]))
print('F-mesure:' , f_measure(wheat_refsets[True], wheat_testsets[True]))

print()

print('Gold:')
print('---------')
print('Precision:', precision(gold_refsets[True], gold_testsets[True]))
print('Recall:'   , recall(gold_refsets[True], gold_testsets[True]))
print('F-mesure:' , f_measure(gold_refsets[True], gold_testsets[True]))

Money-fx:
---------
Precision: 0.3639344262295082
Recall: 0.8283582089552238
F-mesure: 0.5056947608200456

Wheat:
---------
Precision: 0.32098765432098764
Recall: 0.9285714285714286
F-mesure: 0.4770642201834862

Gold:
---------
Precision: 0.5306122448979592
Recall: 0.896551724137931
F-mesure: 0.6666666666666666


## Classifieur multiclasse

In [6]:
def create_multi_dataset(documents, tags, feature_extractor, **kwargs):
    if 'to_lower' in kwargs and kwargs['to_lower']:
        documents = list(map(lambda d: (list(map(str.lower, d[0])), d[1]), documents))

    if 'lemmatizer' in kwargs:
        lemmatizer = kwargs['lemmatizer']
        documents = list(map(lambda d: (list(map(lemmatizer.lemmatize, d[0])), d[1]), documents))
    
    if 'stopwords' in kwargs:
        stopwords = set(kwargs['stopwords'])
        documents = list(map(
            lambda d: (
                list(filter(lambda w: not w.lower() in stopwords and w[0].isalnum(), d[0])), 
                d[1]
            ), documents))
        
    analyzer_res = []
    if 'analyzer' in kwargs:
        analyzer_res = kwargs['analyzer'](documents)

    dataset = []
    for document in documents:
        document_tags = list(set(tags).intersection(document[1]))
        tag = 'other' if document_tags == [] else document_tags[0]

        dataset.append((feature_extractor(document[0], analyzer_res), tag))
    
    shuffle(dataset)
    return dataset

In [7]:
def best_multi_classifier(documents, tag, hyperparams):
    print('Finding best milti-classifier for {}'.format(tag))
    print('----------')

    best = (None, 0.0)
    for hyperparam in hyperparams:
        dataset = create_multi_dataset(documents, tag, **hyperparam)
        train_set, test_set, dev_set = split_dataset(dataset)
        classifier = nltk.NaiveBayesClassifier.train(train_set)
        acc = nltk.classify.accuracy(classifier, dev_set)
        
        if acc > best[1]:
            best = (classifier, acc)
        
        print('Accuracy using "{}": {:.2f}%'.format(hyperparam['title'], acc*100))
    return (best[0], test_set)

### Classification des documents pour `money-fx`, `wheat` et `gold`
Réutilisation des hyperparamètre utilisé dans la partie sur les classifiers binaires

In [8]:
tags = ['money-fx', 'wheat', 'gold']

Nous pouvons constater que le fait de mettre en minuscule améliore systématiquement le score.
La suppression des stopwords améliore notablement les résultats. La lémmatisation ne change pas grand chose aux scores.

In [18]:
classifier_multi, multi_testset = best_multi_classifier(documents, tags, hyperparams)

Finding best classifier for ['money-fx', 'wheat', 'gold']
----------
Accuracy using "To lower: no, Lemmatize: no, No stopwords: no": 81.42%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: no": 79.84%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: no": 80.82%
Accuracy using "To lower: no, Lemmatize: no, No stopwords: yes": 84.43%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: yes": 87.63%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: yes": 85.77%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: no": 81.09%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: yes": 86.14%


In [19]:
multi_refsets, multi_testsets = ref_test_sets(multi_testset, classifier_multi)

### Precision, recall et F-mesure

Nous constatons que la precision est meilleures pour le classifiers multi-class, mais le recall est identique ou inférieur. Cependant la F1 mesure est systèmatiquement meilleur pour le multi-class.


| | | precision | recall | F1 | 
| :--- | :--- | --- | --- | --- | 
| Multi-class | Money-fx | 41.52% | 80.00% | 54.66% |
| Multi-class | Wheat    | 33.54% | 92.98% | 49.30% |
| Multi-class | Gold     | 54.83% | 85.00% | 66.66% |
| Binaire     | Money-fx | 36.39% | 82.83% | 50.56% |
| Binaire     | Wheat    | 32.09% | 92.85% | 47.70% |
| Binaire     | Gold     | 53.06% | 89.65% | 66.66% |


In [20]:
words = tags[:]
print("Score for multiclass classifiers for {}".format(", ".join(map(lambda i: i.title(), words))))
words.append('other')
for word in words:
    print("")
    print('{}:'.format(word.title()))
    print('---------')
    print('Precision:', precision(multi_refsets[word], multi_testsets[word]))
    print('Recall:'   , recall(multi_refsets[word], multi_testsets[word]))
    print('F-mesure:' , f_measure(multi_refsets[word], multi_testsets[word]))


Score for multiclass classifiers for Money-Fx, Wheat, Gold

Money-Fx:
---------
Precision: 0.41522491349480967
Recall: 0.8
F-mesure: 0.5466970387243735

Wheat:
---------
Precision: 0.33544303797468356
Recall: 0.9298245614035088
F-mesure: 0.4930232558139535

Gold:
---------
Precision: 0.5483870967741935
Recall: 0.85
F-mesure: 0.6666666666666666

Other:
---------
Precision: 0.9797619047619047
Recall: 0.8524080787156914
F-mesure: 0.911658820271393
